In [18]:
import os
import numpy as np
from PIL import Image, ImageDraw
import shutil
import csv
import cv2
import torch

from utils.utils0 import tensor_affine_transform, transform_to_displacement_field
from utils.utils1 import transform_points_DVF
from utils.SuperPoint import SuperPointFrontend, PointTracker
superpoint = SuperPointFrontend('utils/superpoint_v1.pth', nms_dist=4,
                          conf_thresh=0.015, nn_thresh=0.7, cuda=True)

# Define parameters
image_size = (256, 256)  # Size of the images
num_samples = 200  # Number of samples for each shape
max_translation = 20  # Maximum translation in pixels
max_rotation = 10  # Maximum rotation in degrees
max_shear = 5  # Maximum shear in degrees
max_scale = 1.1  # Maximum scale factor

# Create a function to generate star images
def generate_star_images(num_samples, output_dir="Dataset/synthetic_shape_dataset", num_images=1):
    # delete all files and subdirectories in the output directory
    if os.path.exists(output_dir):
        shutil.rmtree(output_dir)
    os.makedirs(output_dir, exist_ok=True)

    # Create a csv file to store the affine parameters (training and validation sets)
    affine_parameters_path_train = os.path.join('Dataset', "dataset_shape_synth_train.csv")
    with open(affine_parameters_path_train, "w", newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["source", "target", "M00", "M01", "M02", "M10", "M11", "M12"])
    affine_parameters_path_test = os.path.join('Dataset', "dataset_shape_synth_test.csv")
    with open(affine_parameters_path_test, "w", newline='') as csvfile:
        writer = csv.writer(csvfile)
        writer.writerow(["source", "target", "M00", "M01", "M02", "M10", "M11", "M12"])

    # save 10% of the images for testing
    num_test = int(num_samples * 0.1)
    num_train = num_samples - num_test
    print(f"Generating {num_images*num_train} training images and {num_images*num_test} testing images")
    print(f"Location of the images: {output_dir}")
    print(f"Saving affine parameters to {affine_parameters_path_train} and {affine_parameters_path_test}")

    # create a list to store different point locations
    points_list = []

    # Generate images
    for i in range(num_samples):
        
        # Create a fixed-size image
        image = Image.new("L", image_size, 0)
        draw = ImageDraw.Draw(image)
        
        # Draw a star centered in the image
        center = (image_size[0] // 2, image_size[1] // 2)
        size = np.random.randint(50, min(image_size) // 2)
        points = []
        for _ in range(10):
            angle = np.random.uniform(0, 2 * np.pi)
            x = int(center[0] + size * np.cos(angle))
            y = int(center[1] + size * np.sin(angle))
            points.append((x, y))
        draw.polygon(points, fill=np.random.randint(100, 256))

        # Save original image
        original_image_path = os.path.join(output_dir, f"star_{i}_original.png")
        #cv2.imwrite(original_image_path, image)
        image.save(original_image_path)

        for j in range(num_images):
            # random (2x3) affine transformation matrix
            #M = np.array([[1.0, 0.0, np.random()], [0.0, 1.0, 0.0]])
            rand_range = 0.2
            test_random = np.random.uniform(-rand_range, rand_range, 2)
            M = np.array([[1.0, 0.0, test_random[0]], [0.0, 1.0, test_random[1]]])

            image = np.array(image)
            img_transformed = tensor_affine_transform(torch.tensor(image).unsqueeze(0).unsqueeze(0).float(), 
                                                      torch.tensor(M).unsqueeze(0).float())
            img_transformed = img_transformed.squeeze(0).squeeze(0).numpy()

            # TODO: save heatmaps for other version of network
            points1, desc1, heatmap1 = superpoint(image.astype(np.float32)/255)
            points2, desc2, heatmap2 = superpoint(np.array(img_transformed).astype(np.float32)/255)

            tracker = PointTracker(5, nn_thresh=0.7)
            matches = tracker.nn_match_two_way(desc1, desc2, nn_thresh=0.7)

            matches1 = points1[:2, matches[0, :].astype(int)]
            # matches1 = matches1.T[None, :, :]
            matches2 = points2[:2, matches[1, :].astype(int)]
            
            # add noise to the transformed image and save it
            img_transformed = img_transformed + np.random.randint(-10, 10, size=img_transformed.shape)
            transformed_image_path = os.path.join(output_dir, f"star_{i}_transformed_{j}.png")
            cv2.imwrite(transformed_image_path, np.array(img_transformed))

            # transform the points using the displacement field
            # print(torch.tensor(M)[None, :, :].shape, torch.tensor(image)[None, None, :, :].shape)
            matches1_transformed_DVF = transform_points_DVF(matches1.copy(), 
                torch.tensor(M).view(1, 2, 3), torch.tensor(image)[None, None, :, :])
            # print(f'Img {i}, diff: {matches1_transformed_DVF[:, 0] - matches2[:, 0]}')
            points_list.append(matches1_transformed_DVF[:, 0] - matches2[:, 0])

            # Save affine parameters to a csv file (one line per image pair)
            if i < num_test:
                with open(affine_parameters_path_test, "a", newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow([original_image_path, transformed_image_path, 
                                     M[0, 0], M[0, 1], M[0, 2], M[1, 0], M[1, 1], M[1, 2]])
            else:
                with open(affine_parameters_path_train, "a", newline='') as csvfile:
                    writer = csv.writer(csvfile)
                    writer.writerow([original_image_path, transformed_image_path, 
                                     M[0, 0], M[0, 1], M[0, 2], M[1, 0], M[1, 1], M[1, 2]])
    print("Done!")
    print('MAE point location error:', np.mean(np.abs(np.array(points_list))))
    print()

# Generate star images
output_dir = "Dataset/synthetic_shape_dataset"  # Output directory
generate_star_images(num_samples, output_dir=output_dir, num_images=2)

# Generate star images with 2 transformations
output_dir = "Dataset/synthetic_shape_dataset_multiple"  # Output directory
# generate_star_images(num_samples, output_dir=output_dir, num_images=2)

Generating 360 training images and 40 testing images
Location of the images: Dataset/synthetic_shape_dataset
Saving affine parameters to Dataset/synthetic_shape_dataset/dataset_shape_synth_train.csv and Dataset/synthetic_shape_dataset/dataset_shape_synth_test.csv
Done!
MAE point location error: 0.4179319398359124



In [19]:
# grab images in the directory
# import glob
# import matplotlib.pyplot as plt
# import matplotlib.image as mpimg
# import cv2

# # grab images in the directory
# images = glob.glob('Dataset/synthetic_shape_dataset/*.png')
# len(images)

# # sort images
# images.sort()
# for i in range(len(images)):
#     print(images[i])


In [20]:
# use SIFT to detect and extract features from the image 
# and plot the images in pairs with the keypoints overlaid
# sift = cv2.SIFT_create()

# for i in range(0, 100, 2):
#     fig, ax = plt.subplots(1, 2, figsize=(20, 10))
#     img1 = cv2.imread(images[i], 0)
#     img2 = cv2.imread(images[i+1], 0)
#     kp, des = sift.detectAndCompute(img1, None)
#     img = cv2.drawKeypoints(img1, kp, None)
#     ax[0].imshow(img)
#     ax[0].axis('off')
#     ax[0].set_title(f'#{int(i/2+1)} Original Image')

#     kp, des = sift.detectAndCompute(img2, None)
#     img = cv2.drawKeypoints(img2, kp, None)
#     ax[1].imshow(img)
#     ax[1].axis('off')
#     ax[1].set_title(f'Transformed Image')
#     os.makedirs('Dataset/synthetic_shape_dataset/plot', exist_ok=True)
#     plt.savefig(f'Dataset/synthetic_shape_dataset/plot/{int(i/2+1)}.png')
#     plt.close()

